In [0]:
from google.colab import drive
import keras
from keras import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,LabelEncoder

sns.set()

In [4]:
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
pd.options.display.max_rows=15

# **데이터 전처리**

In [0]:
train = pd.read_csv('/gdrive/My Drive/Titanic/train.csv')
test = pd.read_csv('/gdrive/My Drive/Titanic/test.csv')

train_test_data = [train, test]

In [0]:
for dataset in train_test_data:
    dataset.loc[dataset['Sex']=='male','Sex'] = 0
    dataset.loc[dataset['Sex']=='female','Sex'] = 1

In [8]:
for dataset in train_test_data:
  Ticket_data = dataset["Ticket"].value_counts().sort_index()
  Ticket_list = Ticket_data.index.tolist()
  print(Ticket_list)
  dataset["Ticket_Group"] = 0
  for idx in range(len(dataset)):
    dataset["Ticket_Group"][idx] = Ticket_list.index(dataset["Ticket"][idx])

['110152', '110413', '110465', '110564', '110813', '111240', '111320', '111361', '111369', '111426', '111427', '111428', '112050', '112052', '112053', '112058', '112059', '112277', '112379', '113028', '113043', '113050', '113051', '113055', '113056', '113059', '113501', '113503', '113505', '113509', '113510', '113514', '113572', '113760', '113767', '113773', '113776', '113781', '113783', '113784', '113786', '113787', '113788', '113789', '113792', '113794', '113796', '113798', '113800', '113803', '113804', '113806', '113807', '11668', '11751', '11752', '11753', '11755', '11765', '11767', '11769', '11771', '11774', '11813', '11967', '12233', '12460', '12749', '13049', '13213', '13214', '13502', '13507', '13509', '13567', '13568', '14311', '14312', '14313', '14973', '1601', '16966', '16988', '17421', '17453', '17463', '17464', '17465', '17466', '17474', '17764', '19877', '19928', '19943', '19947', '19950', '19952', '19972', '19988', '19996', '2003', '211536', '21440', '218629', '219533', 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
for dataset in train_test_data:
  dataset['Embarked'] = dataset['Embarked'].fillna('S')
  dataset['Embarked'] = dataset['Embarked'].astype(str)

In [0]:
for dataset in train_test_data:
  dataset['Fare'] = dataset['Fare'].fillna(7.180641176470588)

In [0]:
for dataset in train_test_data:
  dataset.loc[dataset['Age'].isnull()==True,'Age_Nan'] = 1
  dataset.loc[dataset['Age'].isnull()==False,'Age_Nan'] = 0 

# **정규화**

In [0]:
for dataset in train_test_data:
    dataset['Age_Nan'] = StandardScaler().fit_transform(dataset['Age_Nan'].values.reshape(-1, 1))
    dataset['Fare'] = StandardScaler().fit_transform(dataset['Fare'].values.reshape(-1, 1))
    dataset['Pclass'] = StandardScaler().fit_transform(dataset['Pclass'].values.reshape(-1, 1))
    dataset['Sex'] = StandardScaler().fit_transform(dataset['Sex'].values.reshape(-1, 1))
    dataset['SibSp'] = StandardScaler().fit_transform(dataset['SibSp'].values.reshape(-1, 1))
    dataset['Parch'] = StandardScaler().fit_transform(dataset['Parch'].values.reshape(-1, 1))
    dataset['Ticket_Group'] = StandardScaler().fit_transform(dataset['Ticket_Group'].values.reshape(-1, 1))

# **불필요한 열 삭제**

In [0]:
train.drop(['Cabin', 'Age', 'Embarked', 'Ticket', 'Name', 'PassengerId'], axis=1, inplace=True)
test.drop(['Cabin', 'Age', 'Embarked', 'Ticket', 'Name'], axis=1, inplace=True)

In [0]:
target = train['Survived']
train_data = train.drop('Survived', axis=1)

In [15]:
print(test.shape)
print(target.shape)
print(train_data.shape)

(418, 8)
(891,)
(891, 7)


# **TEST**

In [16]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
clf = SVC()
# clf = DecisionTreeClassifier()
clf.fit(train_data, target)
 
test_data = test.drop("PassengerId", axis=1).copy()
prediction = clf.predict(test_data)
submission = pd.DataFrame({
        "PassengerId": test["PassengerId"],
        "Survived": prediction
    })
 
submission.to_csv('/gdrive/My Drive/Titanic/submission_Age_Nan.csv', index=False)
submission = pd.read_csv('/gdrive/My Drive/Titanic/submission_Age_Nan.csv')
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
